In [137]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

import tkinter as tk
from tkinter import filedialog

import statsmodels.api as sm
# from statsmodels.formula.api import ols
# from statsmodels.compat import lzip
# from __future__ import print_function

$$ Price \ Elasticity \ of \ Demand = \frac{\% \ Change \ in \ \color{red}{Quantity} \ \color{white}{Demand} \ of \ Product \ \color{red}{A}}{\% \ Change \ in \ \color{red}{Price} \ \color{white}{of} \ Product \ \color{red}{A}} $$
$$ Cross \ Price \ Elasticity \ of \ Demand = \frac{\% \ Change \ in \ \color{red}{Quantity} \ \color{white}{Demand} \ of \ Product \ \color{red}{A}}{\% \ Change \ in \ \color{red}{Price} \ \color{white}{of} \ Product \ \color{red}{B}} $$

## **Cross Price elasticity of demand**

#### **Competitors/substitutes:**
We would have a **positive** cross-price correlation:
> ## When one product <font color="red">decreases</font> price, another one's quantity sold <font color="red">decreases</font>

#### **Complements:**
Getting a **negtive** cross-price correlation:
> ## When one product <font color="red">decreases</font> price, another one's quantity sold <font color="red">increases</font>



In [99]:
root = tk.Tk()
root.withdraw()
path = filedialog.askopenfilename()

In [192]:
df = pd.read_csv(path, encoding="ISO-8859-1")
df_tbr = df[(df["BRIDGESTONE TYPE"] == "TBR") & (df["DIMENSION (Truck Tyres)"].str.contains("315/80/22.5")) & (df["BRAND"].isin(["MAXXIS", "MICHELIN", "BRIDGESTONE"]))]

In [193]:
df_tbr.columns

Index(['Period', 'Item', 'BRAND', 'REGION', 'BRIDGESTONE TYPE',
       'DIMENSION (Truck Tyres)', 'RIMDIAMET.'/MM', 'WIDTH IN INCH',
       'WIDTH IN MM', 'ASPECT RATIO', 'TUBE / TUBLESS', 'LOADINDEX',
       'FirstActivity', 'SALES UNITS', 'SALES <LC>', 'PRICE TWD/UN.',
       'TBR BRAND', 'LSR2 BRAND', 'TTL BRAND', 'For Report'],
      dtype='object')

In [194]:
df_tbr["Product"] = df_tbr["BRAND"] + " " + df_tbr["Item"] + " " + df_tbr["DIMENSION (Truck Tyres)"]
test1 = df_tbr.groupby(["Period", "Product"]).agg({"PRICE TWD/UN.": "mean", "SALES UNITS": "sum"}).reset_index()
# test1["UNIT PRICE"] = test1["SALES <LC>"] / test1["SALES UNITS"]
x_pivot = test1.pivot(index="Period", columns="Product", values="PRICE TWD/UN.")
y = test1.pivot(index="Period", columns="Product", values="SALES UNITS")


In [195]:
x_value = pd.DataFrame(x_pivot.to_records())
x_value.fillna(method="ffill", inplace=True)
# x_values.dropna(axis=1, inplace=True)
x_value

,Period,BRIDGESTONE G515 315/80/22.5,BRIDGESTONE G582 315/80/22.5,BRIDGESTONE M729 315/80/22.5,BRIDGESTONE M778 315/80/22.5,BRIDGESTONE M798 315/80/22.5,BRIDGESTONE M805 315/80/22.5,BRIDGESTONE M840 315/80/22.5,BRIDGESTONE R150 315/80/22.5,BRIDGESTONE R156 315/80/22.5,BRIDGESTONE R294 315/80/22.5,MAXXIS MA219 315/80/22.5,MAXXIS MA268 315/80/22.5,MAXXIS MD816 315/80/22.5,MAXXIS MD880 315/80/22.5,MAXXIS UL-387 315/80/22.5,MAXXIS UM816 315/80/22.5,MAXXIS UM958 315/80/22.5,MAXXIS UR279 315/80/22.5,MAXXIS UR288 315/80/22.5,MICHELIN X LINE ENERGY Z 315/80/22.5,MICHELIN X MULITWAY XZER 315/80/22.5,MICHELIN X MULTI D 315/80/22.5,MICHELIN X MULTI ENERGY Z 315/80/22.5,MICHELIN X MULTI HD D 315/80/22.5,MICHELIN X MULTI HD Z 315/80/22.5,MICHELIN X MULTI Z 315/80/22.5,MICHELIN X MULTIWAY 3D XDE 315/80/22.5,MICHELIN X MULTIWAY 3D XZE 315/80/22.5,MICHELIN X WORKS HD D 315/80/22.5,MICHELIN X WORKS HD Z 315/80/22.5,MICHELIN XDE2 PLUS 315/80/22.5,MICHELIN XZA2 ENERGY 315/80/22.5,MICHELIN XZA2 PLUS ENERGY 315/80/22.5
0,2019-01-01,13776.0,13960.333333,14750.000000,NaN,NaN,NaN,NaN,14331.000000,NaN,13991.333333,NaN,NaN,11565.333333,NaN,12800.0,12574.500000,11950.000000,13500.0,11964.000000,17500.0,16000.000000,NaN,16018.500000,NaN,NaN,NaN,17833.333333,17869.333333,NaN,NaN,NaN,17800.0,NaN
1,2019-02-01,14119.0,14500.000000,15277.666667,NaN,NaN,16000.000000,NaN,14082.000000,NaN,13796.000000,NaN,12590.500000,12646.000000,NaN,12561.0,12554.750000,12900.000000,13500.0,11898.000000,17000.0,16121.500000,NaN,16200.500000,NaN,NaN,NaN,17981.500000,17301.500000,NaN,NaN,17500.000000,17800.0,NaN
2,2019-03-01,14000.0,14000.000000,14196.666667,NaN,NaN,16000.000000,NaN,14425.250000,NaN,13774.666667,NaN,12500.000000,12817.000000,NaN,12561.0,12442.500000,11980.500000,12250.0,12199.000000,17000.0,16000.000000,NaN,16651.000000,NaN,NaN,NaN,17442.000000,18132.666667,NaN,NaN,17500.000000,13500.0,NaN
3,2019-04-01,14976.5,13255.000000,15279.666667,NaN,NaN,16000.000000,NaN,14450.333333,NaN,14154.666667,NaN,11500.000000,12250.000000,NaN,12561.0,12317.500000,11875.000000,13000.0,11674.666667,16300.0,16000.000000,NaN,16338.000000,NaN,NaN,NaN,18875.000000,17285.333333,NaN,NaN,17500.000000,17500.0,NaN
4,2019-05-01,14750.0,14303.000000,15413.000000,NaN,NaN,16932.000000,14500.0,14093.333333,NaN,14114.666667,NaN,13000.000000,11941.000000,NaN,12561.0,12308.500000,11784.333333,13000.0,11823.333333,17000.0,17042.000000,NaN,17043.333333,NaN,NaN,NaN,20250.000000,17765.000000,NaN,NaN,17500.000000,17200.0,NaN
5,2019-06-01,14000.0,14428.333333,14500.000000,NaN,NaN,15250.000000,14500.0,14258.000000,NaN,14192.333333,NaN,13000.000000,12500.000000,NaN,13000.0,12500.000000,12000.000000,13000.0,11744.666667,17000.0,18000.000000,NaN,15423.333333,NaN,NaN,NaN,17150.000000,17918.333333,NaN,NaN,17500.000000,17200.0,NaN
6,2019-07-01,14000.0,13500.000000,14460.333333,NaN,NaN,16000.000000,14500.0,14581.500000,NaN,13928.500000,NaN,12750.000000,12000.000000,NaN,12800.0,12461.000000,12000.000000,11500.0,11738.333333,17000.0,18000.000000,NaN,16116.000000,NaN,NaN,NaN,17127.500000,17729.666667,NaN,NaN,17000.000000,17100.0,NaN
7,2019-08-01,14400.0,14000.000000,14672.000000,NaN,NaN,15500.000000,14500.0,14971.750000,14000.0,14104.333333,NaN,12500.000000,12000.000000,NaN,12500.0,12453.000000,11750.000000,11500.0,11992.000000,17750.0,18000.000000,NaN,16468.666667,NaN,NaN,NaN,20250.000000,17700.000000,NaN,NaN,17000.000000,17100.0,NaN
8,2019-09-01,13924.0,13890.000000,14678.666667,NaN,NaN,16000.000000,14500.0,14340.333333,14000.0,14250.000000,NaN,10650.000000,13677.000000,NaN,12500.0,11751.333333,12000.000000,13150.0,11706.666667,17750.0,17500.000000,NaN,16451.500000,NaN,NaN,NaN,17900.500000,17400.000000,NaN,NaN,17000.000000,17100.0,NaN
9,2019-10-01,13903.0,13500.000000,15326.000000,NaN,NaN,13000.000000,14500.0,14470.000000,14000.0,13938.333333,NaN,11500.000000,10000.000000,NaN,10000.0,11935.000000,11737.000000,13150.0,11716.333333,18000.0,16000.000000,NaN,16839.333333,NaN,NaN,NaN,17350.000000,17500.000000,NaN,NaN,17

In [196]:
y = y[["BRIDGESTONE R150 315/80/22.5"]]
y.columns = ["R150 sales"]
y = y.reset_index()
y

,Period,R150 sales
0,2019-01-01,1180.0
1,2019-02-01,1061.0
2,2019-03-01,1605.0
3,2019-04-01,1643.0
4,2019-05-01,1057.0
5,2019-06-01,938.0
6,2019-07-01,1130.0
7,2019-08-01,673.0
8,2019-09-01,826.0
9,2019-10-01,567.0


In [197]:
result = pd.merge(x_value, y, on="Period")

In [198]:
result = result.loc[:, ["MICHELIN X MULTI Z 315/80/22.5", "MAXXIS UR288 315/80/22.5", "R150 sales"]]

In [225]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [226]:
ss = MinMaxScaler()
result = pd.DataFrame(ss.fit_transform(result), columns=["MICHELIN X MULTI Z 315/80/22.5", "MAXXIS UR288 315/80/22.5", "R150 sales"])

In [227]:
# result.dropna(inplace=True)
result.fillna(result["MICHELIN X MULTI Z 315/80/22.5"].mean(), inplace=True)

In [228]:
# result.reset_index(drop=True, inplace=True)

In [229]:
x_values = result.loc[:, result.columns[:-1]]
y = result.loc[:, result.columns[-1]]

In [230]:
mean_xvalues = x_values.mean()
mean_yvalues = y.mean()

In [231]:
# If add_constant() doesn't works,  most likely that theres is a column with variance=0, i.e. a column with all identical values.
# The solution is to add has_constant option in the add_constant() function, like: has_constant="add"
X = sm.add_constant(x_values)
# X = sm.add_constant(x_values.loc[:, x_values.columns[-1]])
model = sm.OLS(y, X, missing="drop")
results = model.fit()

results_summry = results.summary()
results_summry

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             R150 sales   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.095
Method:                 Least Squares   F-statistic:                     3.460
Date:                Fri, 10 Feb 2023   Prob (F-statistic):             0.0400
Time:                        16:33:02   Log-Likelihood:                0.48756
No. Observations:                  48   AIC:                             5.025
Df Residuals:                      45   BIC:                             10.64
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                              0.3084      0.113      2.722      0.009       0.080       0.537
MICHELIN X MULTI Z 315/80/22.5    -0.1217      0.209     -0.581      0.564      -0.543       0.300
MAXXIS UR288 315/80/22.5           0.4644      0.177      2.627      0.012       0.108       0.821
==============================================================================
Omnibus:                        4.770   Durbin-Watson:                   1.538
Prob(Omnibus):                  0.092   Jarque-Bera (JB):                4.242
Skew:                           0.728   Prob(JB):                        0.120
Kurtosis:                       3.008   Cond. No.                         7.72
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [232]:
pvalue = results.pvalues

In [233]:
new_df = pd.read_html(results_summry.tables[1].as_html(), header=0, index_col=0)[0]
new_df["p_value"] = pvalue

In [234]:
new_df.index.name = "name"
new_df.reset_index()

,name,coef,std err,t,P>|t|,[0.025,0.975],p_value
0,const,0.3084,0.113,2.722,0.009,0.080,0.537,0.009200
1,MICHELIN X MULTI Z 315/80/22.5,-0.1217,0.209,-0.581,0.564,-0.543,0.300,0.563899
2,MAXXIS UR288 315/80/22.5,0.4644,0.177,2.627,0.012,0.108,0.821,0.011747


In [235]:
new_df["crossprice_elasticity"] = (new_df.coef) * (mean_xvalues / mean_yvalues)
new_df = new_df.reset_index()
pvalue_significant = new_df["p_value"]

In [236]:
new_df["R150_CPE"] = np.where((pvalue_significant > .05), "No effet", new_df["crossprice_elasticity"])
new_df.dropna()

,name,coef,std err,t,P>|t|,[0.025,0.975],p_value,crossprice_elasticity,R150_CPE
1,MICHELIN X MULTI Z 315/80/22.5,-0.1217,0.209,-0.581,0.564,-0.543,0.300,0.563899,-0.143676,No effet
2,MAXXIS UR288 315/80/22.5,0.4644,0.177,2.627,0.012,0.108,0.821,0.011747,0.414563,0.41456283369273395


In [237]:
new_df.set_index("name")

,coef,std err,t,P>|t|,[0.025,0.975],p_value,crossprice_elasticity,R150_CPE
name,,,,,,,,,
const,0.3084,0.113,2.722,0.009,0.080,0.537,0.009200,NaN,nan
MICHELIN X MULTI Z 315/80/22.5,-0.1217,0.209,-0.581,0.564,-0.543,0.300,0.563899,-0.143676,No effet
MAXXIS UR288 315/80/22.5,0.4644,0.177,2.627,0.012,0.108,0.821,0.011747,0.414563,0.41456283369273395


## **Price Elasticity of Demand**

In [284]:
df = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\2021-2026_Size Data_All_20230118.xlsx", None)

In [285]:
df = df["SIZE DATA"]
df["預算/前實績"].unique()

array(['21PY', 'RT3', '22OB', '22ACT', 'RT4', '24MBP', 'RT2', '23OB_V1',
       '22FSCT_V1', '22FSCT_V2', '23OB_V2'], dtype=object)

In [286]:
df["通路明細"].unique()

array(['AG', 'EX', 'FLT-REP', 'FLT-OE', 'OE', 'OT', 'FS', 'CD', 'TK',
       'DS', '其他', '一般店', 'A', 'FMC ', 'MM', '中盤', 'BSSC-FLT', 'FMC',
       'FMC BFP', 'AG花東', 'AG南國', 'AG國隆', 'AG豐正', 'EX-BSAP', 'EX-BSJ',
       'EX-JREP', 'OTHER', '量販店', 'D', 'BSSC', '販社', 'BSAL', 'BSNZ',
       'F HONG KONG', 'BSTK', 'TBSC', 'BSTSG', 'KOREA', 'BSJ',
       'OTHER MEA', 'BSCIS', 'BSAF', 'BATO-NA', 'BSAP', 'FLT', 'FT', '三陽',
       '中華汽車', '台灣本田汽車', '國瑞', '華創車電', '裕隆日產汽車', '順益車輛工業', 'TTC', 'BFP',
       nan], dtype=object)

In [321]:
df = df[(df["預算/前實績"].isin(["21PY", "22ACT"])) & (df["國內市場財別"] == "TBR(G)") & (df["大通路分類"] == "REP")]
df = df[~df["通路明細"].str.contains("AG| FLT| FT| OE")]
data = df.copy()

KeyError: '預算/前實績'

In [334]:
ref_table = pd.read_excel(r"C:\Users\kc.hsu\Python workspace\sales_related\ReferenceTable\20230206_PCODE整理表.xlsx")

In [335]:
ref_table["name"] = ref_table["規格"] + " " + ref_table["花紋"] + " " + ref_table["產地"]
tire_dict = dict(zip(ref_table["PCODE"], ref_table["name"]))

In [336]:
data = data[data["Date"] >= "2022-01-01"]

In [337]:
data["name"] = data["商品代號"].map(tire_dict)

In [338]:
data["Date"] = pd.to_datetime(data["年月"], format="%Y%m")

In [339]:
new_data = data.groupby(["Date", "name"]).agg({"條數": "sum", "NET金額":"sum"}).reset_index()
new_data["avg unit price"] = new_data["NET金額"] / new_data["條數"]
new_data = new_data[["Date", "name", "條數", "avg unit price"]]

In [340]:
new_data = new_data[new_data["條數"] != 0]

In [341]:
x_pivot = new_data.pivot(index="Date", columns="name", values="avg unit price")
x_values = pd.DataFrame(x_pivot.to_records())
x_values.fillna(method="ffill", inplace=True)
x_values

,Date,1000R20 R156 TBSC,1100R20 M840 TBSC,1100R20 R156 BTMT,11R22.5 R156 TBSC,12R22.5 D600 BTMT,12R22.5 F400 BTMT,12R22.5 M729 BTMT,12R22.5 R150 BSJ,12R22.5 R150 BTMT,12R22.5 R150AZ BSCN,12R22.5 R156 BTMT,12R22.5 R224 BTMT,1400R20 R180 BSJ,1400R20 VSJ BSJ,205/75R17.5 R294 BSJ,215/75R17.5 R156 TBSC,215/75R17.5 R184 BSJ,215/75R17.5 R294 BSJ,225/70R19.5 M810 BSJ,225/75R17.5 R294 BSJ,225/90R17.5 M810 BSJ,225/90R17.5 R225 BSJ,235/75R17.5 R156 TBSC,235/75R17.5 R294 BSJ,245/70R17.5 R168 BSJ,245/70R19.5 G623 BSJ,245/70R19.5 M729 BTMT,255/70R22.5 R294 BSJ,265/70R19.5 R294 BTMT,275/70R22.5 R162 BTMT,275/70R22.5 R208 BSCN,275/70R22.5 UAP1 BTMT,285/70R19.5 R184 BSJ,295/80R22.5 M798 BTMT,295/80R22.5 R150 BSJ,295/80R22.5 R150 BTMT,295/80R22.5 R150EZ BSCN,295/80R22.5 R154 BSJ,295/80R22.5 R156 BTMT,295/80R22.5 R192 BSJ,295/80R22.5 R294 BSJ,315/80R22.5 D600 BTMT,315/80R22.5 F400 BTMT,315/80R22.5 G515 BSJ,315/80R22.5 G582 BTMT,315/80R22.5 M729 BTMT,315/80R22.5 M749 BTMT,315/80R22.5 M778Z BSCN,315/80R22.5 M805 BTMT,315/80R22.5 M840 BTMT,315/80R22.5 R150 BSJ,315/80R22.5 R150 BTMT,315/80R22.5 R150BZ BSCN,315/80R22.5 R156 BTMT,315/80R22.5 R294 BTMT,385/65R22.5 R168 BTMT,385/65R22.5 R249 BTMT,750R16 R156 TBSC,750R16 R225 BSJ,825R16 R225 BSJ,9.5R17.5 R294 BSJ,9.5R17.5 R294 TBSC
0,2022-01-01,10543.333333,10505.555556,10102.500000,NaN,6975.142857,7227.529412,9372.814947,9497.771429,9431.360190,8041.820000,10474.5,8673.102564,NaN,NaN,6229.000000,5335.000000,NaN,5671.542918,NaN,6064.482143,5099.580247,4611.429204,5744.413333,6127.946768,6482.545455,5245.846154,NaN,8408.0000,NaN,9432.000000,6900.525926,NaN,NaN,12475.000000,9726.836310,8919.258654,7965.371429,NaN,9155.058659,9048.0,9914.440678,8196.446281,7940.563107,NaN,10231.807229,10631.230409,12629.750000,9400.400000,12213.366667,NaN,10932.441176,10180.890490,8870.313253,NaN,10551.577938,12755.758514,13082.626506,NaN,5105.630058,5632.943299,6425.070822,NaN
1,2022-02-01,10543.333333,12690.333333,12676.500000,NaN,8178.400000,8196.486486,9998.533742,10286.316062,9601.365482,8095.240000,9619.0,9544.067708,21176.15,NaN,5762.250000,5335.000000,NaN,5752.600000,8123.013333,5980.769231,5639.601227,5331.419795,5659.125000,6156.418519,7492.444444,11472.500000,NaN,-7596.5000,NaN,6480.090909,5157.894737,NaN,8493.750000,12467.000000,10375.987745,9667.028855,7943.000000,NaN,7652.918660,9048.0,10371.650000,9511.703030,9145.544041,NaN,10970.444944,11367.844671,12629.750000,9413.706422,13272.803922,NaN,34054.000000,10825.562657,8924.505882,NaN,11242.304615,13256.950192,14554.694444,NaN,5103.991285,5611.411874,6393.315238,NaN
2,2022-03-01,10919.625000,11832.000000,11220.142857,NaN,7426.797872,7504.727273,9524.928803,10203.134615,9621.575092,7663.800000,9619.0,10668.785714,21176.15,NaN,5880.837037,5528.516667,NaN,5972.163588,8123.013333,6247.844037,5361.889610,5033.553785,5876.361702,6340.285714,8839.629310,5389.000000,NaN,-7596.5000,NaN,8965.354286,4895.833333,10937.4,9914.666667,13773.100000,10107.593625,9419.665635,8088.535865,NaN,9328.378882,9048.0,10135.513889,8590.670846,8564.258182,NaN,10958.791866,11054.853901,14075.000000,9385.102151,13196.568182,13413.00,10710.333333,10555.514030,9126.414948,NaN,10928.649049,13116.215768,13583.080808,NaN,5262.295580,5821.911168,6637.095064,6617.895425
3,2022-04-01,10836.000000,12199.000000,11495.625000,11848.923077,7426.797872,7232.240000,10112.577869,10276.694444,9473.394822,8652.380000,10286.0,10176.500000,21176.15,NaN,5992.147410,5127.000000,NaN,5987.073048,8123.013333,6363.294118,6427.733333,6063.240000,5847.263889,6407.800000,7213.895833,6642.666667,NaN,-7596.5000,NaN,8709.124324,5500.000000,10937.4,9914.666667,13319.200000,10147.488889,9739.340726,7988.501859,NaN,9072.672727,9524.0,12590.500000,8590.670846,8212.125000,NaN,11537.792793,11379.401734,14075.000000,8969.099010,12000.500000,14486.00,12655.732673,10946.383578,8579.864706,NaN,10924.524823,12984.765766,15647.944444,NaN,5472.075676,5951.790000,7041.058366,6713.439394
4,2022-05-01,11185.166667,12199.000000,13112.8000

In [342]:
y_pivot = new_data.pivot(index="Date", columns="name", values="條數")
y_values = pd.DataFrame(y_pivot.to_records())
y_values.fillna(method="ffill", inplace=True)
y_values

,Date,1000R20 R156 TBSC,1100R20 M840 TBSC,1100R20 R156 BTMT,11R22.5 R156 TBSC,12R22.5 D600 BTMT,12R22.5 F400 BTMT,12R22.5 M729 BTMT,12R22.5 R150 BSJ,12R22.5 R150 BTMT,12R22.5 R150AZ BSCN,12R22.5 R156 BTMT,12R22.5 R224 BTMT,1400R20 R180 BSJ,1400R20 VSJ BSJ,205/75R17.5 R294 BSJ,215/75R17.5 R156 TBSC,215/75R17.5 R184 BSJ,215/75R17.5 R294 BSJ,225/70R19.5 M810 BSJ,225/75R17.5 R294 BSJ,225/90R17.5 M810 BSJ,225/90R17.5 R225 BSJ,235/75R17.5 R156 TBSC,235/75R17.5 R294 BSJ,245/70R17.5 R168 BSJ,245/70R19.5 G623 BSJ,245/70R19.5 M729 BTMT,255/70R22.5 R294 BSJ,265/70R19.5 R294 BTMT,275/70R22.5 R162 BTMT,275/70R22.5 R208 BSCN,275/70R22.5 UAP1 BTMT,285/70R19.5 R184 BSJ,295/80R22.5 M798 BTMT,295/80R22.5 R150 BSJ,295/80R22.5 R150 BTMT,295/80R22.5 R150EZ BSCN,295/80R22.5 R154 BSJ,295/80R22.5 R156 BTMT,295/80R22.5 R192 BSJ,295/80R22.5 R294 BSJ,315/80R22.5 D600 BTMT,315/80R22.5 F400 BTMT,315/80R22.5 G515 BSJ,315/80R22.5 G582 BTMT,315/80R22.5 M729 BTMT,315/80R22.5 M749 BTMT,315/80R22.5 M778Z BSCN,315/80R22.5 M805 BTMT,315/80R22.5 M840 BTMT,315/80R22.5 R150 BSJ,315/80R22.5 R150 BTMT,315/80R22.5 R150BZ BSCN,315/80R22.5 R156 BTMT,315/80R22.5 R294 BTMT,385/65R22.5 R168 BTMT,385/65R22.5 R249 BTMT,750R16 R156 TBSC,750R16 R225 BSJ,825R16 R225 BSJ,9.5R17.5 R294 BSJ,9.5R17.5 R294 TBSC
0,2022-01-01,9.0,9.0,10.0,NaN,7.0,34.0,281.0,140.0,211.0,50.0,2.0,156.0,NaN,NaN,4.0,10.0,NaN,466.0,NaN,56.0,162.0,226.0,75.0,263.0,99.0,26.0,NaN,10.0,NaN,96.0,135.0,NaN,NaN,3.0,336.0,1040.0,70.0,NaN,358.0,32.0,59.0,242.0,103.0,NaN,332.0,855.0,4.0,10.0,30.0,NaN,204.0,1388.0,166.0,NaN,417.0,323.0,83.0,NaN,346.0,388.0,1059.0,NaN
1,2022-02-01,9.0,9.0,2.0,NaN,25.0,37.0,326.0,193.0,197.0,100.0,17.0,192.0,20.0,NaN,32.0,14.0,NaN,95.0,75.0,13.0,163.0,293.0,16.0,270.0,72.0,2.0,NaN,2.0,NaN,55.0,38.0,NaN,32.0,8.0,408.0,1109.0,50.0,NaN,209.0,20.0,80.0,165.0,193.0,NaN,445.0,882.0,4.0,109.0,51.0,NaN,2.0,1987.0,85.0,NaN,650.0,261.0,36.0,NaN,459.0,539.0,1621.0,NaN
2,2022-03-01,8.0,17.0,7.0,NaN,94.0,66.0,309.0,104.0,273.0,100.0,17.0,112.0,20.0,NaN,135.0,60.0,NaN,379.0,75.0,109.0,154.0,251.0,94.0,133.0,116.0,2.0,NaN,2.0,NaN,175.0,48.0,5.0,18.0,10.0,251.0,969.0,237.0,NaN,322.0,20.0,72.0,319.0,275.0,NaN,418.0,705.0,10.0,186.0,44.0,11.0,21.0,1675.0,388.0,NaN,473.0,241.0,99.0,NaN,362.0,394.0,1094.0,153.0
3,2022-04-01,2.0,4.0,8.0,13.0,94.0,50.0,244.0,36.0,309.0,100.0,1.0,94.0,20.0,NaN,251.0,1.0,NaN,397.0,75.0,17.0,15.0,25.0,72.0,25.0,48.0,6.0,NaN,2.0,NaN,185.0,16.0,5.0,18.0,10.0,45.0,496.0,269.0,NaN,165.0,30.0,24.0,319.0,32.0,NaN,111.0,346.0,10.0,101.0,14.0,1.0,101.0,816.0,170.0,NaN,423.0,222.0,18.0,NaN,185.0,200.0,514.0,66.0
4,2022-05-01,6.0,4.0,5.0,13.0,8.0,60.0,260.0,53.0,311.0,100.0,2.0,138.0,20.0,NaN,43.0,5.0,NaN,375.0,75.0,263.0,137.0,148.0,5.0,227.0,46.0,8.0,NaN,8.0,NaN,208.0,16.0,5.0,18.0,9.0,114.0,552.0,29.0,NaN,152.0,10.0,54.0,47.0,35.0,1.0,217.0,453.0,8.0,84.0,27.0,1.0,867.0,1281.0,525.0,24.0,451.0,211.0,38.0,10.0,313.0,328.0,457.0,248.0
5,2022-06-01,5.0,8.0,4.0,13.0,4.0,104.0,444.0,102.0,450.0,100.0,2.0,280.0,2.0,NaN,147.0,67.0,NaN,481.0,75.0,404.0,77.0,118.0,77.0,401.0,144.0,8.0,NaN,8.0,NaN,201.0,16.0,5.0,18.0,32.0,193.0,724.0,20.0,NaN,146.0,8.0,62.0,122.0,64.0,123.0,322.0,634.0,4.0,48.0,41.0,4.0,97.0,1447.0,240.0,24.0,504.0,282.0,45.0,23.0,431.0,558.0,1442.0,302.0
6,2022-07-01,13.0,2.0,1.0,2.0,4.0,18.0,251.0,25.0,409.0,50.0,2.0,112.0,30.0,NaN,196.0,6.0,NaN,201.0,9.0,281.0,36.0,42.0,33.0,208.0,15.0,15.0,NaN,16.0,NaN,121.0,16.0,5.0,18.0,6.0,32.0,342.0,204.0,NaN,211.0,28.0,70.0,115.0,200.0,61.0,69.0,489.0,66.0,48.0,22.0,2.0,20.0,1341.0,97.0,24.0,335.0,278.0,46.0,4.0,262.0,169.0,392.0,1.0
7,2022-08-01,13.0,21.0,1.0,2.0,4.0,18.0,221.0,25.0,271.0,50.0,2.0,78.0,20.0,NaN,108.0,1.0,NaN,398.0,9.0,210.0,44.0,104.0,18.0,166.0,90.0,23.0,NaN,16.0,2.0,269.0,16.0,5.0,18.0,10.0,205.0,917.0,40.0,12.0,174.0,12.0,41.0,38.0,116.0,86.0,133.0,811.0,66.0,48.0,19.0,2.0,451.0,1554.0,160.0,24.0,309.0,168.0,52.0,29.0,269.0,169.0,560.0,339.0
8,2022-09-01,7.0,3.0,1.0,2.0,4.0,18.0,206.0,13.0,238.0,174.0,2.0,156.0,20.

In [347]:
points = []
results_values = {
    "name": [],
    "price_elasticity": [],
    "price_mean": [],
    "quantity_mean": [],
    "intercept": [],
    "t_score":[],
    "slope": [],
    "coefficient_pvalue" : [],
    "rsquared": []
}


for column in x_values.columns[1:]:
    column_points = []
    for i in range(len(x_values[column])):
        if not np.isnan(x_values[column][i]) and not np.isnan(y_values[column][i]):
            column_points.append((x_values[column][i], y_values[column][i]))
    df = pd.DataFrame(list(column_points), columns=["x_values", "y_values"])
    # print(df)
    x_value = df["x_values"]
    y_value = df["y_values"]
    X = sm.add_constant(x_value)
    model = sm.OLS(y_value, X)
    result = model.fit()
    
    if result.f_pvalue < 0.05:
        rsquared = result.rsquared
        coefficient_pvalue = result.f_pvalue
        intercept, slope = result.params
        mean_price = np.mean(x_value)
        mean_quantity = np.mean(y_value)
        tintercept, t_score = result.tvalues
        
        price_elasticity = (slope) * (mean_price/mean_quantity)

        results_values["name"].append(column)
        results_values["price_elasticity"].append(price_elasticity)
        results_values["price_mean"].append(mean_price)
        results_values["quantity_mean"].append(mean_quantity)
        results_values["intercept"].append(intercept)
        results_values['t_score'].append(t_score)
        results_values["slope"].append(slope)
        results_values["coefficient_pvalue"].append(coefficient_pvalue)
        results_values["rsquared"].append(rsquared)
        
final_df = pd.DataFrame.from_dict(results_values)
df_elasticity = final_df[['name','price_elasticity','t_score','coefficient_pvalue','slope','price_mean','quantity_mean','intercept','rsquared']]
df_elasticity

,name,price_elasticity,t_score,coefficient_pvalue,slope,price_mean,quantity_mean,intercept,rsquared
0,12R22.5 D600 BTMT,-8.630788,-2.553605e+00,2.868608e-02,-0.022411,8568.657383,22.250000,214.285024,0.394706
1,12R22.5 R156 BTMT,-40.085467,-1.407143e+01,6.450859e-08,-0.017147,10325.208333,4.416667,181.460812,0.951924
2,1400R20 R180 BSJ,-4.463646,-3.227938e+00,1.035681e-02,-0.005055,20228.345455,22.909091,125.167164,0.536550
3,225/70R19.5 M810 BSJ,-8.612951,-2.148268e+01,4.827871e-09,-0.038642,9017.006061,40.454545,388.887551,0.980872
4,225/90R17.5 M810 BSJ,-1.958665,-2.317988e+00,4.291962e-02,-0.024093,6706.929173,82.500000,244.089883,0.349512
5,225/90R17.5 R225 BSJ,-1.481367,-2.240095e+00,4.899631e-02,-0.029446,6552.537989,130.250000,323.198106,0.334134
6,255/70R22.5 R294 BSJ,0.287933,5.098251e+00,4.652132e-04,0.000557,4480.310417,8.666667,6.171251,0.722162
7,265/70R19.5 R294 BTMT,-5.162902,-1.038505e+15,1.968995e-45,-0.005248,8460.400000,8.600000,53.000954,1.000000
8,275/70R22.5 R208 BSCN,5.252638,2.635418e+00,2.492927e-02,0.032860,5754.521166,36.000000,-153.094976,0.409870
9,285/70R19.5 R184 BSJ,-3.985887,-1.310889e+01,3.614507e-07,-0.008440,9617.462121,20.363636,101.530790,0.950233
